Импортируем необходимые библиотеки

In [3]:
import os
import pandas as pd
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

Зададим основные параметры

In [4]:
batch_size = 64
num_classes = 10
epochs = 20
learning_rate = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

При необходимости загружаем датасет, разбиваем на тренировочную и валидационную выборки

In [5]:
need_download = not os.path.exists('data')
train = torchvision.datasets.CIFAR10(root = 'data', train = True, download = need_download, transform = ToTensor())
dataset = DataLoader(train, batch_size = batch_size, shuffle = True)
test = torchvision.datasets.CIFAR10(root = 'data', train = False, download = need_download, transform = ToTensor())
dataset_test = DataLoader(test, batch_size = batch_size, shuffle = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


Функция для тренировки модели

In [6]:
def basic_training(model, allow_print=True) -> nn.Module:
    if allow_print:
        print(f"{model.__class__.__name__} is training")
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr = learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        for batch in dataset:
            data, target = batch
            data, target = data.to(device), target.to(device)
            opt.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            opt.step()
        if allow_print:
            print(f"Epoch: {epoch}, Loss: {loss.item()}")
    if allow_print:
        print("Training done")
    return model

Функция для тестирования модели

In [7]:
def basic_testing(model, allow_print=True) -> float:
    if allow_print:
        print(f"{model.__class__.__name__} is testing")
    model.to(device)
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch in dataset_test:
            data, target = batch
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    if allow_print:
        print(f"Accuracy: {correct / len(test) * 100}%")
    return correct / len(test)

Модель только на полносвязных слоях

In [8]:
class DenseModel(torch.nn.Module):
    def __init__(self):
        super(DenseModel, self).__init__()
        IMG_SIZE = 32 * 32 * 3
        self.fc1 = nn.Linear(IMG_SIZE, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.selu(self.fc2(x))
        x = self.fc3(x)
        return x

Пробуем обучить и оценить качество модели

In [12]:
model = DenseModel()
model = basic_training(model)
basic_testing(model)

DenseModel is training
Epoch: 0, Loss: 1.5844886302947998
Epoch: 1, Loss: 1.780474305152893
Epoch: 2, Loss: 1.7519259452819824
Epoch: 3, Loss: 1.5110762119293213
Epoch: 4, Loss: 2.0993995666503906
Epoch: 5, Loss: 1.7433979511260986
Epoch: 6, Loss: 1.2541842460632324
Epoch: 7, Loss: 1.8724783658981323
Epoch: 8, Loss: 1.0642768144607544
Epoch: 9, Loss: 1.1619980335235596
Epoch: 10, Loss: 1.761088252067566
Epoch: 11, Loss: 1.5877214670181274
Epoch: 12, Loss: 1.6626615524291992
Epoch: 13, Loss: 1.321022629737854
Epoch: 14, Loss: 1.5918039083480835
Epoch: 15, Loss: 1.6306958198547363
Epoch: 16, Loss: 1.0055201053619385
Epoch: 17, Loss: 1.6507893800735474
Epoch: 18, Loss: 1.122137188911438
Epoch: 19, Loss: 0.9554914236068726
Training done
DenseModel is testing
Accuracy: 49.75%


0.4975

Модель со сверточными слоями

In [18]:
class ModelConvolutionalLayers(torch.nn.Module):
    def __init__(self):
        super(ModelConvolutionalLayers, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.fc = nn.Linear(4 * 4 * 128, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


Пробуем обучить и оценить качество модели

In [19]:
model = ModelConvolutionalLayers()
basic_training(model)
basic_testing(model)

ModelConvolutionalLayers is training
Epoch: 0, Loss: 1.38761568069458
Epoch: 1, Loss: 1.5851092338562012
Epoch: 2, Loss: 1.3749301433563232
Epoch: 3, Loss: 1.2786552906036377
Epoch: 4, Loss: 1.1772303581237793
Epoch: 5, Loss: 1.7018786668777466
Epoch: 6, Loss: 0.7055850625038147
Epoch: 7, Loss: 1.208823561668396
Epoch: 8, Loss: 1.239600419998169
Epoch: 9, Loss: 1.0998029708862305
Epoch: 10, Loss: 1.5608346462249756
Epoch: 11, Loss: 0.962348997592926
Epoch: 12, Loss: 0.9363788962364197
Epoch: 13, Loss: 1.3520491123199463
Epoch: 14, Loss: 0.8267087936401367
Epoch: 15, Loss: 1.4121803045272827
Epoch: 16, Loss: 1.1779121160507202
Epoch: 17, Loss: 0.9294607043266296
Epoch: 18, Loss: 0.47261950373649597
Epoch: 19, Loss: 1.0281343460083008
Training done
ModelConvolutionalLayers is testing
Accuracy: 65.36999999999999%


0.6537

Сравниваем модели с различным количеством слоев и нейронов в слоях

In [20]:
class ModelSmallAmountLayers(torch.nn.Module):
    def __init__(self):
        super(ModelSmallAmountLayers, self).__init__()
        self.fc = nn.Linear(32 * 32 * 3, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

Пробуем обучить и оценить качество модели

In [21]:
model = ModelSmallAmountLayers()
basic_training(model)
basic_testing(model)

ModelSmallAmountLayers is training
Epoch: 0, Loss: 1.6779299974441528
Epoch: 1, Loss: 1.6097882986068726
Epoch: 2, Loss: 1.9150526523590088
Epoch: 3, Loss: 2.012221097946167
Epoch: 4, Loss: 1.6962625980377197
Epoch: 5, Loss: 1.630715012550354
Epoch: 6, Loss: 1.8835877180099487
Epoch: 7, Loss: 1.5686497688293457
Epoch: 8, Loss: 1.9710755348205566
Epoch: 9, Loss: 2.0606210231781006
Epoch: 10, Loss: 2.136298656463623
Epoch: 11, Loss: 1.8353599309921265
Epoch: 12, Loss: 1.6801728010177612
Epoch: 13, Loss: 1.9019471406936646
Epoch: 14, Loss: 1.7235713005065918
Epoch: 15, Loss: 1.5795425176620483
Epoch: 16, Loss: 1.8054556846618652
Epoch: 17, Loss: 1.495120882987976
Epoch: 18, Loss: 1.5526131391525269
Epoch: 19, Loss: 1.7700011730194092
Training done
ModelSmallAmountLayers is testing
Accuracy: 39.73%


0.3973

Можно сделать вывод о том, что модель с всего лишь одним слоем показывает существенно худшие результаты в сравнении с, к примеру, моделью со сверточными слоями. В общем случае, чем более многослойна модель и чем больше нейронов содержится в каждом ее слое, тем более сложные -- с точки зрения разделимости -- объекты она способна классифицировать, однако в месте с тем ростут и требования к ресурсам

Дропаут

In [22]:
class ModelDropout(torch.nn.Module):
    def __init__(self):
        super(ModelDropout, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Dropout(0.2),
            nn.Flatten(),
            nn.Linear(4 * 4 * 128, 128),
            nn.ReLU(),
        )
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

Пробуем обучить и оценить качество модели

In [23]:
model = ModelDropout()
model = basic_training(model)
basic_testing(model)

ModelDropout is training
Epoch: 0, Loss: 1.7339071035385132
Epoch: 1, Loss: 1.7028706073760986
Epoch: 2, Loss: 1.8868827819824219
Epoch: 3, Loss: 1.2097877264022827
Epoch: 4, Loss: 2.032628297805786
Epoch: 5, Loss: 1.6308537721633911
Epoch: 6, Loss: 1.5603388547897339
Epoch: 7, Loss: 2.409684181213379
Epoch: 8, Loss: 1.495227575302124
Epoch: 9, Loss: 1.415284514427185
Epoch: 10, Loss: 2.0286433696746826
Epoch: 11, Loss: 2.0475454330444336
Epoch: 12, Loss: 1.2859288454055786
Epoch: 13, Loss: 2.292361259460449
Epoch: 14, Loss: 1.1931170225143433
Epoch: 15, Loss: 1.7853786945343018
Epoch: 16, Loss: 0.6332534551620483
Epoch: 17, Loss: 1.1680864095687866
Epoch: 18, Loss: 1.3080027103424072
Epoch: 19, Loss: 1.273888349533081
Training done
ModelDropout is testing
Accuracy: 60.28%


0.6028